In [11]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.models import load_model
import datetime
from scipy.io import wavfile
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pydub import AudioSegment
from scipy.signal import resample

In [12]:
from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

ImportError: cannot import name 'tensorflow_backend' from 'keras.backend' (C:\Users\User\anaconda3\lib\site-packages\keras\backend.py)

In [ ]:
# 데이터 가져오기
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
# 결측치 제거
train = train.dropna()
test = test.dropna()
train.head()
test.head()

In [ ]:
# 반응변수 전처리
def preprocess_y(data, exchange_player=False):
    y = data.drop_duplicates(['game_id', 'winner']).winner.reset_index(drop=True)
    # 기존의 값을 스왑하여 저장
    if (exchange_player == True):
        y = y.append(-(y - 1)).reset_index(drop=True)
    return y

In [ ]:
# 설명변수 전처리
def preprocess_X(data, exchange_player=False):
    print("start")
    # game_id 개수만큼의 index를 가진 DataFrame X 생성
    n = data.game_id.max() + 1
    X = pd.DataFrame(index=range(n)[data.game_id.min():])
    
    print("time")
    # time 변수
    X['time'] = data.drop_duplicates(['game_id'],keep='last').set_index('game_id').time
    # 시간을 초로 맞춘다  분             초
    X['time'] = (X.time*100//100*60 + X.time*100%100)#.astype(int)
    
    print("species")
    # species 더미 변수
    X = pd.concat([pd.get_dummies(data[data.player == 0].drop_duplicates(['game_id']).set_index('game_id').species).rename(columns={'P':'0_protoss','T':'0_terran','Z':'0_zerg'}),
                pd.get_dummies(data[data.player == 1].drop_duplicates(['game_id']).set_index('game_id').species).rename(columns={'P':'1_protoss','T':'1_terran','Z':'1_zerg'}),
                X],axis=1)
    
    print("event_count")
    # event 카운트 (플레이어별 event 개수)
    contents = data.loc[:,['player','game_id','time']].groupby(['player', 'game_id']).count().unstack(level=0)
    contents.columns = ['0_event', '1_event']
    X['0_event'], X['1_event'] = contents['0_event'], contents['1_event']

    print("event_count/time")
    # event 카운트 / time
    X['0_event_per_sec'], X['1_event_per_sec'] = X['0_event'] /X.time, X['1_event'] /X.time

    print("each_event_count")
    # event == Ability, AddToControlGroup, Camera, ControlGroup, GetControlGroup, Right Click, Selection, SetControlGroup일 때 각각 카운트
    contents = data.loc[:,['player','event','game_id','time']].groupby(['player', 'event', 'game_id']).count().unstack(level=[0,1]).fillna(0).astype(int)
    contents.columns = ['0_'+x for x in sorted(data.event.unique())] + ['1_'+x for x in sorted(data.event.unique())]
    for i in contents.columns:
        X[i] = contents[i]

    # event == Camera일 때 event_contents의 2차원 좌표 간 euclidean distance sum, min, median, max
    def move_sum(i):
        return sum(np.sqrt(np.diff(i.map(lambda x: x[4:x.find(',')]).astype(float)) **2 +
                       np.diff(i.map(lambda x: x[x.find(',')+2:len(x)-1]).astype(float)) **2))
    def move_min(i):
        if len(i) == 1:
            return 0
        return min(np.sqrt(np.diff(i.map(lambda x: x[4:x.find(',')]).astype(float)) **2 +
                       np.diff(i.map(lambda x: x[x.find(',')+2:len(x)-1]).astype(float)) **2))
    def move_median(i):
        if len(i) == 1:
            return 0
        return np.median(np.sqrt(np.diff(i.map(lambda x: x[4:x.find(',')]).astype(float)) **2 + 
                             np.diff(i.map(lambda x: x[x.find(',')+2:len(x)-1]).astype(float)) **2))
    def move_max(i):
        if len(i) == 1:
            return 0
        return max(np.sqrt(np.diff(i.map(lambda x: x[4:x.find(',')]).astype(float)) **2 +
                       np.diff(i.map(lambda x: x[x.find(',')+2:len(x)-1]).astype(float)) **2))
    
    print("camera_event")
    contents = (data[data.event == 'Camera'].loc[:,['player','game_id','event_contents']].
              groupby(['player','game_id'])).agg([move_sum,move_min,move_median,move_max]).unstack(level=0)
    contents.columns = [y+x for x in ['sum','min','median','max'] for y in ['0_move_','1_move_']]
    for i in tqdm(contents.columns):
        X[i] = contents[i].fillna(0)

    # 30초 이내 move_sum
    print("camera_event < 30")
    contents = (data[(data.time < 0.3) & (data.event == 'Camera')].loc[:,['player','game_id','event_contents']].
              groupby(['player','game_id'])).agg(move_sum).unstack(level=0)
    contents.columns = ['0_move_sum_30sec','1_move_sum_30sec']
    for i in tqdm(contents.columns):
        X[i] = contents[i]

    print("ability_event")
    # event == Ability일 때 event_contents 더미 변수 생성, 카운트
    contents = pd.DataFrame(data.event_contents[(data.event == 'Ability')].map(lambda x: x[x.find('(')+1:x.find(')')]))  # event_contents의 16진수 코드만 추출
    contents['game_id'], contents['player'], contents['count'] = data.game_id, data.player, 1
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_','1_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in tqdm(contents_X.columns):
        X[i] = contents_X[i]
        X[i] = X[i].fillna(0).astype(int)

    # event == Ability일 때 event_contents 더미 변수 생성 / time
    print("ability_event/time")
    for i in tqdm(contents_X.columns):
        X[i+'_div_time'] = X[i] /X.time

    # event == Selection일 때 event_contents 더미 변수 생성, 카운트
    print("selection_event_count")
    contents = data[data.event == 'Selection'].event_contents.map(lambda x: re.sub('\s\[.....\]', '', re.sub('\s\[......\]', '', re.sub('\s\[.......\]', '', x))).
                                                                replace('[', '').replace(']', '').replace(' ', '').replace('\'', ''))
    contents = contents.str.split(',')
    max_num = max(contents.map(lambda x: len(x)))
    t = [0 for x in range(max_num)]
    for i in tqdm(range(max_num)):
        t[i] = pd.DataFrame(contents[contents.map(lambda x: len(x) > i)].map(lambda x: x[i]))
    contents = pd.concat([t[i] for i in range(max_num)])
    contents['game_id'], contents['player'], contents['count'] = data.game_id, data.player, 1
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_','1_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in tqdm(contents_X.columns):
        X[i] = contents_X[i]
        X[i] = X[i].fillna(0).astype(int)

    # event == Selection일 때 event_contents 더미 변수 생성 / time
    print("selection_event/time")
    for i in tqdm(contents_X.columns):
        X[i+'_div_time'] = X[i] /X.time

    # 30초 이내 event == Selection일 때 event_contents 더미 변수 생성, 카운트
    print("selection_event < 30")
    contents = data[(data.time < 0.3) & (data.event == 'Selection')].event_contents.map(lambda x: re.sub('\s\[.....\]', '', re.sub('\s\[......\]', '', re.sub('\s\[.......\]', '', x))).
                                                                                      replace('[', '').replace(']', '').replace(' ', '').replace('\'', ''))
    contents = contents.str.split(',')
    max_num = max(contents.map(lambda x: len(x)))
    t = [0 for x in range(max_num)]
    for i in tqdm(range(max_num)):
        t[i] = pd.DataFrame(contents[contents.map(lambda x: len(x) > i)].map(lambda x: x[i]))
    contents = pd.concat([t[i] for i in range(max_num)])
    contents['game_id'], contents['player'], contents['count'] = data.game_id, data.player, 1
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_','1_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in tqdm(contents_X.columns):
        X[i+'_30sec'] = contents_X[i]
        X[i+'_30sec'] = X[i+'_30sec'].fillna(0).astype(int)

    # event == Right Click일 때 Target 이름 더미 변수 생성, 카운트
    print("rigthClick_event")
    contents = pd.DataFrame(data.event_contents[(data.event == 'Right Click') & (data.event_contents.map(lambda x: str(x)[:6]) == 'Target')].map(lambda x: x[x.find(':')+2:x.find(' [')]))  # event_contents의 Target만 추출
    contents['game_id'], contents['player'], contents['count'] = data.game_id, data.player, 1
    contents_X = pd.DataFrame(columns=[x+y for x in ['0_Target_','1_Target_'] for y in contents.event_contents.unique()])
    contents = contents.groupby(['player','event_contents','game_id']).count().unstack(level=[0,1])
    contents.columns = contents.columns.map(lambda x: str(x[1])+'_Target_'+x[2])
    contents_X = pd.concat([contents_X, contents])
    for i in tqdm(contents_X.columns):
        X[i] = contents_X[i]
        X[i] = X[i].fillna(0).astype(int)

    # 컬럼 이름 순서로 정렬
    print("sort")
    X = X[sorted(X.columns)]

    # player 0,1 자리 바꾼 X1생성, X와 행 병합해 데이터 2배로 만들기
    print("double")
    if (exchange_player == True):
        c = X.shape[1]//2
        X1 = X.copy()
        X1.columns = list(X.columns[c:2*c])+list(X.columns[:c])+['time']
        X1.index = [x+n for x in range(n)]
        X = pd.concat([X, X1])

    return X

In [ ]:
# 데이터 전처리 과정
# 종족을 숫자로 구분
def species_converter(string):
    if string == 'T':
        return 0
    elif string == 'P':
        return 1
    elif string == 'Z':
        return 2
    else:
        raise ValueError

# 데이터 전처리   데이터, Y값 필요 여부
def data_preparation(df, answer=False):
    game_ids = df['game_id'].unique()
    events = ['Ability', 'AddToControlGroup', 'Camera', 'ControlGroup', 'GetControlGroup', 'Right Click', 'Selection', 'SetControlGroup']
    unique_event_0, unique_event_1, delta_event = {}, {}, {}
    for event in events:
        unique_event_0['P0_' + event] = 0
        unique_event_1['P1_' + event] = 0
        delta_event['delta_' + event] = 0
    
    # 각 경기별 및 플레이어별로 종족 그룹화
    species = df.groupby(['game_id', 'player']).species.unique()
#     print(species)
#     game_id  player
#     0        0         [T]
#              1         [T]
    
    # 각 경기별 및 플레이어별로 이벤트 그룹화
    event_count = df.groupby(['game_id', 'player']).event.value_counts()
#     print(event_count)
#     game_id  player  event      
#     0        0       Camera         444
#                      Selection       50
#                      Right Click     35
#                      Ability         34
    
    # 경기별 승자 (Player 1의 승리 확률 : y_data)
    if answer:
        winners = df.groupby(['game_id']).winner.max()
#         print(winners)
#         game_id
#         0        1
#         7        0
    
    x_data, y_data = [], []
    for game_id in tqdm(game_ids):
        # 데이터 통합
        df_event_count = event_count[game_id].unstack(level=-1)
        df = pd.DataFrame(species[game_id])
        df = pd.concat([df, df_event_count], axis=1)   
        df = df.fillna(0)
#         print(df)
#         player    species  Ability  Camera  Right Click  Selection                                                
#         0          [T]       34     444           35         50
#         1          [T]       34     425           28         57

        # 각 플레이어별 종족은 숫자로 변환
        df_P0_species = pd.DataFrame([species_converter(df.loc[0]['species'][0])], columns=['P0_species'])        
        df_P1_species = pd.DataFrame([species_converter(df.loc[1]['species'][0])], columns=['P1_species'])
        # String 종족값 제거
        df = df.drop(['species'], axis=1)
        
        # 플레이어 P0 분류
        df_P0_event = unique_event_0.copy()
        for column in df.columns:
            df_P0_event['P0_' + column] = df.loc[0][column]
        df_P0_event = pd.DataFrame(pd.Series(df_P0_event)).T
#         print(df_P0_event)
#         P0_Ability  P0_AddToControlGroup  P0_Camera  P0_ControlGroup  P0_GetControlGroup  P0_Right Click  P0_Selection  P0_SetControlGroup 
#           34                0               444             0               0                  35             50               0 
       
        # 플레이어 P1 분류
        df_P1_event = unique_event_1.copy()
        for column in df.columns:
            df_P1_event['P1_' + column] = df.loc[1][column]
        df_P1_event = pd.DataFrame(pd.Series(df_P1_event)).T
#         print(df_P1_event)
#         P1_Ability  P1_AddToControlGroup  P1_Camera  P1_ControlGroup  P1_GetControlGroup  P1_Right Click  P1_Selection  P1_SetControlGroup  
#             34                0               425           0               0                  28            57                0  

        # P0 - P1 이벤트
        df_delta_event = delta_event.copy()
        for column in df.columns:
            df_delta_event['delta_' + column] = df_P0_event['P0_' + column][0] - df_P1_event['P1_' + column][0]
        df_delta_event = pd.DataFrame(pd.Series(df_delta_event)).T
#         print(df_delta_event)
#         delta_Ability  delta_AddToControlGroup  delta_Camera  delta_ControlGroup  delta_GetControlGroup  delta_Right Click  delta_Selection
#               0                  0                   19              0                       0                  7                -7 

        # 데이터 통합    P0 종족 숫자값  P0 이벤트   P1 종족 숫자값  P1 이벤트   P0 - P1 이벤트
        out = pd.concat([df_P0_species, df_P0_event, df_P1_species, df_P1_event, df_delta_event], axis=1)
        out.index = [game_id]
        out.index.name = 'game_id'
#         print(out)
#         P0_species  P0_Ability  P0_AddToControlGroup  P0_Camera  \                                     
#            0          34                  0              444   
#         P0_ControlGroup  P0_GetControlGroup  P0_Right Click  P0_Selection  \                                                                     
#               0                   0              35            50   
#         P0_SetControlGroup  P1_species  ...  P1_Selection  \                 
#               0                  0      ...      57   
#         P1_SetControlGroup  delta_Ability  delta_AddToControlGroup  \                
#                0              0                        0   
#         delta_Camera  delta_ControlGroup  delta_GetControlGroup  \          
#            19                   0                      0   
#         delta_Right Click  delta_Selection  delta_SetControlGroup
#                7               -7                      0  
#         [1 rows x 26 columns]
        
        # x_data 대입
        x_data.append(out)
        
        # y_data 대입
        if answer:
            y_data.append(winners[game_id])  

    x_data = pd.concat(x_data)
    y_data = np.array(y_data)
    
    return x_data, y_data

In [ ]:
x_data, y_data = data_preparation(train, answer=True)
x_data.head()

In [ ]:
y_data = preprocess_y(train, True)
x_data = preprocess_X(train, True)
test_x_data = preprocess_X(test, False)

In [ ]:
print(x_data)
print(y_data)
print(test_x_data)

In [ ]:
# 몇 퍼센트로 나눌지 결정
row = round(x_data.shape[0] * 0.7)
print("len :", x_data.shape[0])
print("row :", row)
x_data = np.array(x_data)
y_data = np.array(y_data)
# train_data
# test_data
x_train = x_data[:row,:]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
y_train = y_data[:row]
x_test = x_data[row:,:]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
y_test = y_data[row:]

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# # 모델 생성
model = Sequential()

model.add(LSTM(64, return_sequences=True, input_shape=(x_train.shape[1],1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
# mse, binary_crossentropy
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
model.summary()

# 트레이닝 값으로 학습 # validation_split=0.2 validation_data=(x_test, y_test)
hist = model.fit(x_train, y_train, validation_split=0.2
                 , batch_size=10, epochs=10)
results = model.evaluate(x_test, y_test)

In [ ]:
# 모델 학습 과정 표시하기
fig, loss_ax = plt.subplots()
loss_ax.plot(hist.history['loss'], 'y', label='train_loss')
loss_ax.plot(hist.history['acc'], 'r', label='train_acc')
loss_ax.plot(results[0], 'y', label='test_loss')
loss_ax.plot(results[1], 'r', label='test_acc')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')
plt.show()

In [ ]:
# 모델 저장
model.save('weight2.h5')

In [ ]:
# 모델 로드
model = Sequential()
model = load_model('weight1.h5')

In [ ]:
# test_data 로드 및 전처리
test = pd.read_csv('./data/test.csv')
test_x_data, _ = data_preparation(test, answer=False)

In [ ]:
test_x_data = np.array(test_x_data)
test_x_data = np.reshape(test_x_data, (test_x_data.shape[0], test_x_data.shape[1], 1))
pred = model.predict(test_x_data)
pred = np.reshape(pred, (pred.shape[0]))
pred

In [ ]:
plt.hist(pred)
plt.show()

In [ ]:
# 예측값 저장
submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

submission['winner'] = submission['winner'] + pred
submission.to_csv('./result/submission.csv')
submission.head()